In [12]:
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.page import Page
import os
from datetime import datetime
from dotenv import load_dotenv
import time
import pandas as pd
import re
from facebook_business.exceptions import FacebookRequestError


In [2]:
load_dotenv()  # This loads the variables from .env

True

In [3]:
# Initialize the Facebook API
FacebookAdsApi.init(access_token=os.getenv('FACEBOOK_ACCESS_TOKEN'),
                    app_id=os.getenv('FACEBOOK_APP_ID'),
                    app_secret=os.getenv('FACEBOOK_APP_SECRET'))

In [4]:
def get_page_access_tokens(page_ids):
    page_access_tokens = {}
    for page_id in page_ids:
        try:
            page = Page(page_id)
            fields = ['access_token']
            page_info = page.api_get(fields=fields)
            page_access_tokens[page_id] = page_info['access_token']
        except Exception as e:
            print(f"Error fetching token for page {page_id}: {e}")
    return page_access_tokens

In [13]:
def get_relevant_lead_forms(page_access_tokens):
    relevant_lead_forms = {}
    for page_id, access_token in page_access_tokens.items():
        try:
            # Initialize the API with the page access token
            FacebookAdsApi.init(access_token=access_token)

            # Constructing the Graph API endpoint
            endpoint = f'{page_id}/leadgen_forms'
            params = {
                'fields': 'id,name,created_time',
                'access_token': access_token
            }

            # Make the API call
            response = FacebookAdsApi.get_default_api().call('GET', (endpoint,), params=params)
            lead_forms = response.json()['data']

            evergreen_forms = [form for form in lead_forms if 'Evergreen 2024' in form['name']]

            # Sort evergreen forms by creation time, pick the most recent one
            if evergreen_forms:
                most_recent_form = max(evergreen_forms, key=lambda x: datetime.strptime(x['created_time'], '%Y-%m-%dT%H:%M:%S+0000'))
                relevant_lead_forms[page_id] = (most_recent_form['id'], 'evergreen')
            elif lead_forms:
                # If no evergreen forms, pick the most recent form
                most_recent_form = max(lead_forms, key=lambda x: datetime.strptime(x['created_time'], '%Y-%m-%dT%H:%M:%S+0000'))
                relevant_lead_forms[page_id] = (most_recent_form['id'], 'other')
            else:
                # No lead forms found
                relevant_lead_forms[page_id] = (None, 'none')
                
        except Exception as e:
            print(f"Error fetching lead forms for page {page_id}: {e}")
            relevant_lead_forms[page_id] = (None, 'error')

    return relevant_lead_forms

In [6]:
account_info = pd.read_csv('/Users/lroberts/Downloads/audience_ids_20240123.csv')

In [7]:
def convert_id_columns_to_string(df):
    for col in df.columns:
        if 'id' in col and df[col].dtype == 'float64':
            df[col] = df[col].apply(lambda x: None if pd.isna(x) else str(int(x)))
    return df

In [8]:
account_info = convert_id_columns_to_string(account_info)

In [9]:
account_info.head()

,Account name,Ad Account ID,Budget,utm_Next Month Free,utm_Free Class,utm_Quit Free,utm_Presales,utm_Half Off First Month,utm_2 For 20,utm_2 Free,utm_Free Week,utm_Free Month,utm_Transformation Challenge,utm_$24 Month,broad_id,wepe_id,reach_id,version_consistency,page_id,instagram_account_id
0,"Orangetheory Bedminster, NJ (Regional)",act_284726299130471,"$1,000.00",utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,120203358555740368,120203358547950368,120203358522850368,Consistent,528717637296388,1013155728773139
1,"Orangetheory Madison, AL",act_633860747073903,"$1,500.00",utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,23850812879850444,23850812878870444,23850599656060444,Inconsistent,237950873389015,1564304426944075
2,"Orangetheory Starkville, MS",act_348958262391635,$700.00,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,23858163257200668,23858163253060668,23857879244270668,Consistent,450103698744682,1996513270420017
3,"Orangetheory Bel-Air, MD",act_477143946095880,"$2,500.00",utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,None,None,None,NaN,None,None
4,"Orangetheory Jersey City, NJ",act_1229642834071405,$750.00,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,utm_medium=psoc&utm_source=fb&utm_term=drive&u...,None,None,None,NaN,None,None


In [10]:
# Example usage with the first three page IDs
page_ids = account_info['page_id'][0:3]
page_access_tokens = get_page_access_tokens(page_ids)


In [14]:
relevant_lead_forms = get_relevant_lead_forms(page_access_tokens)
print(relevant_lead_forms)

{'528717637296388': ('934658267749937', 'other'), '237950873389015': ('705430667610893', 'other'), '450103698744682': ('2157958517728369', 'other')}
